In [17]:
import os
import re
import json
import pytz
import string
import pandas as pd
from typing import List

---

### Dados da B3 (Dados das Empresas Listadas)

**Colunas de interesse**

- `tickers`
- `nome_pregao`
- `setor_economico`


In [2]:
tickers = ['B3SA3', 'EQTL3', 'ITUB4', 'PETR4', 'VALE3']
tickers

['B3SA3', 'EQTL3', 'ITUB4', 'PETR4', 'VALE3']

In [3]:
with open('./data/results-b3.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_b3 = pd.DataFrame(data)
df_b3.sample(5)

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
263,GUARARAPES CONFECCOES S.A.,GUARARAPES,NDA,[GUAR3],Consumo Cíclico,Comércio,Tecidos. Vestuário e Calçados,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
423,BRASILAGRO - CIA BRAS DE PROP AGRICOLAS,BRASILAGRO,NM,[AGRO3],Consumo não Cíclico,Agropecuária,Agricultura,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
52,Terra Santa Propriedades Agricolas S.A.,TERRASANTAPA,NM,[LAND3],Consumo não Cíclico,Agropecuária,Agricultura,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
235,ITAUSA S.A.,ITAUSA,N1,"[ITSA3, ITSA4]",Financeiro,Intermediários Financeiros,Bancos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
136,PETTENATI S.A. INDUSTRIA TEXTIL,PETTENATI,NDA,"[PTNT3, PTNT4]",Consumo Cíclico,Tecidos. Vestuário e Calçados,Fios e Tecidos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


In [4]:
df_final = pd.DataFrame()

for index, row in df_b3.iterrows():
    for ticker in row['tickers']:
        if ticker in tickers:
            df_final = df_final.append(row)

In [5]:
df_final.sort_values('razao_social')

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
450,B3 S.A. - BRASIL. BOLSA. BALCÃO,B3,NM,[B3SA3],Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
296,EQUATORIAL ENERGIA S.A.,EQUATORIAL,NM,[EQTL3],Utilidade Pública,Energia Elétrica,Energia Elétrica,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
236,ITAU UNIBANCO HOLDING S.A.,ITAUUNIBANCO,N1,"[ITUB3, ITUB4, ITUB99]",Financeiro,Intermediários Financeiros,Bancos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
138,PETROLEO BRASILEIRO S.A. PETROBRAS,PETROBRAS,N2,"[PETR3, PETR4]",Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
31,VALE S.A.,VALE,NM,[VALE3],Materiais Básicos,Mineração,Minerais Metálicos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


In [6]:
nomes_pregao = df_final.nome_pregao.values.tolist()
nomes_pregao

['VALE', 'PETROBRAS', 'ITAUUNIBANCO', 'EQUATORIAL', 'B3']

In [7]:
ticker_name = {
    'B3SA3': 'B3',
    'EQTL3': 'EQUATORIAL',
    'ITUB4': 'ITAUUNIBANCO',
    'PETR4': 'PETROBRAS',
    'VALE3': 'VALE'
}

---

### Dados do Portal de Notícias Suno Research

In [ ]:
with open('../crawlers/suno/results/suno-petr4.json', encoding='utf8') as json_file:
    data = json.load(json_file) 
    
df_suno = pd.DataFrame(data)

In [ ]:
# Convert date to datetime format
df_suno['date'] = pd.to_datetime(df_suno['date'], format='%d/%m/%Y %H:%M')

# Set date column as index
df_suno.set_index('date', inplace=True)

# Order by date
df_suno.sort_index(inplace=True)

In [32]:
df_suno.head()

,topic,title,search_date,url,tags
date,,,,,
2020-07-16 08:27:00,Negócios,Agenda do Dia: JHSF; Tim; Oi; Petrobras; Metal...,2022-05-03 16:22:46,https://www.suno.com.br/noticias/agenda-do-dia...,"[agenda do dia, JHSF (JHSF3), Metalfrio (FRIO3..."
2020-07-16 09:46:00,Negócios,Petrobras (PETR4) conclui venda de campos pelo...,2022-05-03 16:22:45,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"
2020-07-16 17:28:00,Mercado,"Ibovespa hoje cai 1,22% em mau humor com bater...",2022-05-03 16:22:45,https://www.suno.com.br/noticias/ibovespa-fech...,"[Ibovespa, JHSF (JHSF3), Petrobras (PETR4), Va..."
2020-07-16 19:04:00,Negócios,Petrobras (PETR4): exportação de petróleo para...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/petrobras-pet...,"[China, PETR3, Petrobras (PETR4), Petróleo]"
2020-07-17 09:06:00,Negócios,Agenda do Dia: Moura Dubeux; Tenda; Petrobras;...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/agenda-do-dia...,"[agenda do dia, Cia Hering (HGTX3), Moura Dube..."


---

### Rule-Based Method for NER

- `Suno Research`
  - Do nothing (for now) since the news portal already filter by ticker in the crawler
- `Twitter`
  - Store original tweet and mentioned ticker

#### Suno Research

In [34]:
def get_word_list(sentence: str) -> List[str]:
    list_of_words = [[w.upper().translate(str.maketrans('', '', string.punctuation))] for w in sentence.split()]
    return list_of_words

def get_tag_list(tag_list: List[str]) -> List[List[str]]:
    list_of_tags = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in tag_list]
    return list_of_tags

In [82]:
tz = pytz.timezone('America/Sao_Paulo')

df_suno_t = pd.DataFrame()

for index, row in df_suno.iterrows():

    title = str(row['title'])
    tags = row['tags']

    # Lista de palavras, sem pontuação, com letras maiúsculas
    list_of_words = get_word_list(title)

    # Lista de tags, sem pontuação, com letras maiúsculas
    list_of_tags = get_tag_list(tags)

    # [TAGS]
    company_names_set = set()

    # Flat tag list(e.g. [['AGENDA', 'DO', 'DIA'], ['JHSF', 'JHSF3']] -> ['AGENDA', 'DO', 'DIA', 'JHSF', 'JHSF3'])
    tags_flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_tags for item in sublist]

    # 1. Verifica ocorrência exata ou do nome do pregão na lista de TAGS
    for tag_word in tags_flat_list:
        if (tag_word in tickers):
            company_names_set.add(ticker)
        if (tag_word in nomes_pregao):
            position = list(ticker_name.values()).index(tag_word)
            company_names_set.add(list(ticker_name.keys())[position])

    # Remover pontuação
    word_flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]
    
    if 'VALE' in word_flat_list:
        word_flat_list.remove('VALE')

    # [TITLE]
    # 2. Verifica ocorrência exata ou do nome do pregão no TITLE
    for title_word in word_flat_list:
        if title_word in nomes_pregao:          
            position = list(ticker_name.values()).index(title_word)
            company_names_set.add(list(ticker_name.keys())[position])

    mentioned_tickers = set(word_flat_list).intersection(tickers)
    mentioned_tickers.update(company_names_set)

    if mentioned_tickers and ('VALE3' in mentioned_tickers):
        print (f'Headline: {str(row.title)}')
        for ticker in list(mentioned_tickers):
            print (f'(datetime, ticker, topic) values ({index.replace(tzinfo=tz)}, {ticker}, {row.topic})') 
        print ('-'*35)

Headline: BNDES deve começar a venda de ações da carteira pela Vale (VALE3)
(datetime, ticker, topic) values (2020-08-03 21:31:00-03:00, VALE3, Negócios)
(datetime, ticker, topic) values (2020-08-03 21:31:00-03:00, CXSE3, Negócios)
-----------------------------------


---

#### Twitter

1. Get all tweets from the following list of Twitter Users:

- `@Fatosdomercado`
- `@arkoadvice`
- `@BloombergBrasil`
- `@valorinveste`
- `@InvestingBrasil`
- ...

2. Extract mentions of listed companies with Rule-Based Method for NER
3. Store into separated `pd.DataFrame` with original tweet data and mentioned ticker

In [25]:
def load_tweets(username: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Funtion to load Tweets from Specific Username (stored locally).
    """

    with open(f'../crawlers/twitter/results_related/{username}.json', encoding='utf8') as json_file:
        data = json.load(json_file) 
        
    df_twitter = pd.DataFrame(data)

    # Convert date to datetime format
    df_twitter['date'] = pd.to_datetime(df_twitter['created_at'], format='%Y-%m-%d %H:%M:%S')

    # Set date column as index
    df_twitter.set_index('date', inplace=True)

    # Order by date
    df_twitter.sort_index(inplace=True)

    # Filter Date
    df_twitter = df_twitter.loc[start_date:end_date]

    return df_twitter

In [26]:
def rule_based_ner(tweets_df: pd.DataFrame, words_to_remove: List[str]) -> pd.DataFrame:
    """
    Function to Identify Company Mentions by Word Match (Company Name or Ticker).
    """
    
    df_final = pd.DataFrame()

    for index, row in tweets_df.iterrows():
        
        title = str(row['text'])

        # Lista de palavras, sem pontuação, com letras maiúsculas
        list_of_words = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in title.split()]

        for index, words in enumerate(list_of_words):
            if words and (words[0] in words_to_remove):
                del list_of_words[index]

        if not list_of_words:
            # Skips blank tweets
            continue 

        # Remover pontuação
        flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]

        # Verifica ocorrência exata
        mentioned_tickers = set()
        mentioned_tickers = set(flat_list).intersection(tickers)
        
        for title_word in flat_list:
            if title_word in nomes_pregao:
                position = list(ticker_name.values()).index(title_word)
                company_ticker = list(ticker_name.keys())[position]
                mentioned_tickers.add(company_ticker)

        # Store all info if
        # any occurence was identified
        if mentioned_tickers:
            for ticker in list(mentioned_tickers):
                row['ticker'] = ticker
                df_final = df_final.append(row, ignore_index=True)

    return df_final

In [27]:
# Remove Sentences Starting With
remove_words = ['HTTP', 'HTTPS']

# Select Username
username = 'BloombergBrasil'

# Search Directory
directory = '../crawlers/twitter/results_related/'

df_twitter_final = pd.DataFrame()

frames = list()

# Loop over files in the directory
for filename in os.scandir(directory):
    
    username = filename.name.replace('.json', '')

    # Original Tweets
    df_twitter = load_tweets(username=username, start_date='2018-01-01', end_date='2022-07-14')

    # Original Tweets + Ticker
    df_twitter_t = rule_based_ner(
        tweets_df=df_twitter, 
        words_to_remove=remove_words
    )

    df_twitter_final = df_twitter_final.append(df_twitter_t, ignore_index=True)

In [28]:
df_twitter_final.sample(10)

,tweet_id,created_at,search_dt,text,user_id,screen_name,rt_count,favorite_count,ticker
669,1455936710295101442,2021-11-03 13:35:36,2022-08-02 00:49:07,"Atualização das 13h30: Vale, Petrobras, Cosan ...",70575051,bussoladoinves,0.0,0.0,PETR4
2236,1352217497865510913,2021-01-21 08:32:29,2022-08-02 00:57:56,IPO da Cruzeiro do Sul pode levantar mais de R...,1346759748188831747,InvestidoresO,0.0,0.0,B3SA3
133,1501603972594950150,2022-03-09 14:01:00,2022-08-02 00:49:36,B3 apresenta um caso interessante de crescimen...,25503983,acionistacombr,0.0,1.0,B3SA3
1462,1539261941318836224,2022-06-21 12:00:19,2022-08-02 00:48:57,"Olá, investidores!\nConfiram as empresas que d...",70575051,bussoladoinves,0.0,0.0,PETR4
796,1467975898355884036,2021-12-06 18:55:02,2022-08-02 00:49:05,Confira as empresas que distribuíram ou vão di...,70575051,bussoladoinves,0.0,1.0,B3SA3
140,1507039789719199750,2022-03-24 14:00:59,2022-08-02 00:49:35,Tudo sobre a Vale (VALE3): Tire suas dúvidas a...,25503983,acionistacombr,0.0,0.0,VALE3
674,1456685592343945216,2021-11-05 15:11:23,2022-08-02 00:49:07,Atenção para os novos horários da Bolsa a part...,70575051,bussoladoinves,0.0,0.0,B3SA3
1302,1516092533486542860,2022-04-18 13:33:22,2022-08-02 00:48:59,"Atualização das 13h30: CSN, Vale, BR Malls, JH...",70575051,bussoladoinves,0.0,0.0,VALE3
2375,1451631660064202759,2021-10-22 16:28:52,2022-08-02 00:57:53,Petrobras deve pagar Royalties superiores a R$...,1346759748188831747,InvestidoresO,0.0,0.0,PETR4
748,1463565197529034752,2021-11-24 14:48:29,2022-08-02 00:49:06,A escultura causou burburinho e até foi alvo d...,70575051,bussoladoinves,0.0,0.0,B3SA3


In [32]:
df_twitter_final.ticker.value_counts()

PETR4    1330
B3SA3     936
VALE3     649
ITUB4     160
EQTL3      56
Name: ticker, dtype: int64

In [33]:
df_twitter_final.to_csv('./df_twitter_related_users_with_mentions.csv', sep=';', encoding='utf8')